In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy BiqQuery ML Model on Vertex AI Model Registry and make predictions

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This tutorial demonstrates how to train a model with BigQuery ML and upload it on Vertex AI Model Registry, then make batch predictions.

Learn more about [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction) and Learn more about [BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml).

### Objective

In this tutorial, you learn how to use `Vertex AI Model Registry` with `BigQuery ML` and make batch predictions:

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Model Registry`
- `Vertex AI Endpoint` resources
- `Vertex Explainable AI`
- `BigQuery ML`


The steps performed include:

- Train a model with `BigQuery ML`
- Upload the model to `Vertex AI Model Registry` 
- Create a `Vertex AI Endpoint` resource
- Deploy the `Model` resource to the `Endpoint` resource
- Make `prediction` requests to the model endpoint
- Run `batch prediction` job on the `Model` resource 


### Dataset

The dataset used for this tutorial is the Penguins dataset from <a href="https://cloud.google.com/bigquery/public-data" target="_blank">BigQuery public datasets</a>. This version of the dataset is used to predict the species of penguins from the available features like culmen-length, flipper-depth etc.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery ML

Learn about <a href="https://cloud.google.com/vertex-ai/pricing" target="_blank">Vertex AI
pricing</a> and <a href="https://cloud.google.com/bigquery/pricing" target="_blank">BigQuery pricing</a>, and use the <a href="https://cloud.google.com/products/calculator/" target="_blank">Pricing
Calculator</a>
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.
**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to <a href="https://cloud.google.com/python/setup" target="_blank">Setting up a Python development
environment</a> and the <a href="https://jupyter.org/install" target="_blank">Jupyter
installation guide</a> provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. <a href="https://cloud.google.com/sdk/docs/" target="_blank">Install and initialize the Cloud SDK.</a>

1. <a href="https://cloud.google.com/python/setup#installing_python" target="_blank">Install Python 3.</a>

1. <a href="https://cloud.google.com/python/setup#installing_and_using_virtualenv" target="_blank">Install
   virtualenv</a>
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install the following packages required to execute this notebook. 

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform google-cloud-bigquery pyarrow {USER_FLAG} -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.4 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.1 which is incompatible.
tensorflow-transform 0.14.0 requires protobuf<4,>=3.7, but you have protobuf 4.22.1 which is incompatible.
tensorflow-metadata 0.14.0 requires protobuf<4,>=3.7, but you have protobuf 4.22.1 which is incompatible.
tensorboardx 2.6 requires protobuf<4,>=3.8.0, but you have protobuf 4.22.1 which is incompatible.
google-cloud-vision 3.4.0 requires google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0dev,>=1.34.0, but you have goo

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. <a href="https://console.cloud.google.com/cloud-resource-manager" target="_blank">Select or create a Google Cloud project</a>. When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. <a href="https://cloud.google.com/billing/docs/how-to/modify-project" target="_blank">Make sure that billing is enabled for your project</a>.

1. <a href="https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,bigquery" target="_blank">Enable the Vertex AI and BigQuery APIs</a>. 

1. If you are running this notebook locally, you will need to install the <a href="https://cloud.google.com/sdk" target="_blank">Cloud SDK</a>.

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you can get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "ds-training-380514"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "ds-training-380514":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: ds-training-380514


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You might not be able to use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/general/locations" target="_blank">Vertex AI regions</a>.

In [4]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [5]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated.
**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the <a href="https://console.cloud.google.com/apis/credentials/serviceaccountkey" target="_blank">Create service account key page</a>.

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Import libraries

In [6]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI and Big Query SDKs for Python for your project and corresponding bucket.

In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bqclient = bigquery.Client(project=PROJECT_ID)

## BigQuery ML introduction

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification and regression, in BigQuery using SQL syntax.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs" target="_blank">BigQuery ML documentation</a>.

### BigQuery table used for training

In [8]:
# Define BigQuery table to be used for training

BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### Create BigQuery dataset resource
First, you create an empty dataset resource in your project.

In [9]:
BQ_DATASET_NAME = "penguins" + UUID
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}"""

job = bqclient.query(DATASET_QUERY)
job.result()
print(job.state)

DONE


## Train BigQuery ML model and upload it to Vertex AI Model Registry
Next, you create and train a `BigQuery  ML` tabular regression model from the public dataset penguins and store the model in your project `Vertex AI Model Registry` using the `CREATE MODEL` statement. The model configuration is specified in the `OPTIONS` statement as follows:

- `model_type`: The type and archictecture of tabular model to train, e.g., LOGISTIC_REG.

- `labels`: The column which are the labels.

- `model_registry`: To register a BigQuery ML model to Vertex AI Model Registry, you must use `model_registry="vertex_ai"`.

During this stage, if you are using a supported model type, BQML exports metadata associated to the model to Vertex so it can run Vertex Explainable AI. Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/vertex-xai" target="_blank">supported BQML model types</a>.

Learn more about the <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create" target="_blank">CREATE MODEL statement</a>.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">Managing BigQuery ML models in the Vertex AI Model Registry</a>.

In [10]:
# Write the query to create Big Query ML model

MODEL_NAME = "penguins-lr" + UUID
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='LOGISTIC_REG',
    labels = ['species'],
    model_registry='vertex_ai'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

### Create BigQuery ML Model
Create the BigQuery ML model using the query above and the BigQuery client that you created previously:

In [11]:
# Run the model creation query using BigQuery client

job = bqclient.query(MODEL_QUERY)
print(f"Job state: {job.state}\nJob Error:{job.errors}")

Job state: RUNNING
Job Error:None


Check the job status:

In [12]:
job.result()
print(job.state)

DONE


### Find the model in the Vertex Model Registry

You can use the `Vertex AI Model()` method with `model_name` parameter to find the automatically registered model.

In [13]:
model = aiplatform.Model(model_name=MODEL_NAME)

print(model.gca_resource)

name: "projects/354621994428/locations/us-central1/models/penguins-lr9zk74b5b"
version_id: "1"
version_aliases: "default"
version_create_time {
  seconds: 1680080197
  nanos: 568498000
}
version_update_time {
  seconds: 1680080257
  nanos: 419387000
}
display_name: "penguins-lr9zk74b5b"
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1680080197
  nanos: 568498000
}
update_time {
  seconds: 1680080201
  nanos: 790669000
}
explanation_spec {
  parameters {
    sampled_shapley_attribution {
      path_count: 5
    }
  }
  metadata {
    inputs {
      key: "sex"
      value {
        input_tensor_name: "sex:0"

## Deploy Vertex AI Model resource to a Vertex AI Endpoint resource
You must deploy a model to an `endpoint` before that model can be used to serve online predictions; deploying a model associates physical resources with the model so it can serve online predictions with low latency. 

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_deploy_model_custom_trained_model_sample-python" target="_blank">Deploy a model using the Vertex AI API</a>


### Create a Vertex AI Endpoint resource

If you are deploying a model to an existing endpoint, you can skip this cell.

- `display_name`: Display name for the endpoint.
- `project`: The project ID on which you are creating an endpoint.
- `location`: The region where you are using Vertex AI.

In [14]:
ENDPOINT_DISPLAY_NAME = "bqml-lr-model-endpoint" + UUID

endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint
Create Endpoint backing LRO: projects/354621994428/locations/us-central1/endpoints/2263846063079161856/operations/9187358426840170496
Endpoint created. Resource name: projects/354621994428/locations/us-central1/endpoints/2263846063079161856
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/354621994428/locations/us-central1/endpoints/2263846063079161856')
bqml-lr-model-endpoint9zk74b5b
projects/354621994428/locations/us-central1/endpoints/2263846063079161856


### Deploy the Vertex AI Model resource to Vertex AI Endpoint resource

In [15]:
DEPLOYED_NAME = "bqml-lr-penguins"

model.deploy(endpoint=endpoint, deployed_model_display_name=DEPLOYED_NAME)

print(model.display_name)
print(model.resource_name)

Deploying model to Endpoint : projects/354621994428/locations/us-central1/endpoints/2263846063079161856
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/354621994428/locations/us-central1/endpoints/2263846063079161856/operations/2251788612410540032
Endpoint model deployed. Resource name: projects/354621994428/locations/us-central1/endpoints/2263846063079161856
penguins-lr9zk74b5b
projects/354621994428/locations/us-central1/models/penguins-lr9zk74b5b


## Send prediction request to the Vertex AI Endpoint resource

Now that your Vertex AI Model resource is deployed to a Vertex AI `Endpoint` resource, you can do online predictions by sending prediction requests to the `Endpoint` resource.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models" target="_blank">Get online predictions from custom-trained models</a>

In [16]:
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

prediction = endpoint.predict([instance])
print(prediction)

Prediction(predictions=[{'species_probs': [0.004134393613856876, 8.70873646769593e-06, 0.9958568976496754], 'predicted_species': ['Adelie Penguin (Pygoscelis adeliae)'], 'species_values': ['Chinstrap penguin (Pygoscelis antarctica)', 'Gentoo penguin (Pygoscelis papua)', 'Adelie Penguin (Pygoscelis adeliae)']}], deployed_model_id='8626365810274729984', model_version_id='1', model_resource_name='projects/354621994428/locations/us-central1/models/penguins-lr9zk74b5b', explanations=None)


## Send Explainable AI request to the Vertex AI Endpoint resource

If this is a supported BQML model type for Vertex Explainable AI, you can get explainations for your resource. To learn more about using Vertex AI Explainability on BQML models, see [Explainable AI for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/vertex-xai)


In [17]:
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

explanation = endpoint.explain([instance])
print(explanation)

Prediction(predictions=[{'species_probs': [0.004134393613856876, 8.70873646769593e-06, 0.9958568976496754], 'species_values': ['Chinstrap penguin (Pygoscelis antarctica)', 'Gentoo penguin (Pygoscelis papua)', 'Adelie Penguin (Pygoscelis adeliae)'], 'predicted_species': ['Adelie Penguin (Pygoscelis adeliae)']}], deployed_model_id='8626365810274729984', model_version_id=None, model_resource_name=None, explanations=[attributions {
  baseline_output_value: 29.911764759909278
  instance_output_value: 5.7104316784578835
  feature_attributions {
    struct_value {
      fields {
        key: "sex"
        value {
          number_value: -0.69595153932667131
        }
      }
      fields {
        key: "island"
        value {
          number_value: -0.54527906531066606
        }
      }
      fields {
        key: "flipper_length_mm"
        value {
          number_value: -13.266837681708891
        }
      }
      fields {
        key: "culmen_length_mm"
        value {
          number_v

## Batch Prediction on the BigQuery ML model
Here you request batch predictions directly from the BigQuery ML model; you don't need to deploy the model to an endpoint. For data types that support both batch and online predictions, use batch predictions when you don't require an immediate response and want to process accumulated data by using a single request.

Learn more abount <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">The ML.PREDICT function</a>

In [18]:
sql_ml_predict = f"""SELECT * FROM ML.PREDICT(MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`, 
(SELECT
      *
    FROM
      `{BQ_TABLE}` LIMIT 10))"""

job = bqclient.query(sql_ml_predict)
prediction_result = job.result().to_arrow().to_pandas()

In [19]:
display(prediction_result.head())

,predicted_species,predicted_species_probs,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,37.3,16.8,192.0,3000.0,FEMALE
1,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,46.9,16.6,192.0,2700.0,FEMALE
2,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,39.8,19.1,184.0,4650.0,MALE
3,Adelie Penguin (Pygoscelis adeliae),[{'label': 'Adelie Penguin (Pygoscelis adeliae...,Adelie Penguin (Pygoscelis adeliae),Dream,40.2,20.1,200.0,3975.0,MALE
4,Chinstrap penguin (Pygoscelis antarctica),[{'label': 'Chinstrap penguin (Pygoscelis anta...,Chinstrap penguin (Pygoscelis antarctica),Dream,49.5,19.0,200.0,3800.0,MALE


## Cleaning up

To clean up all Google Cloud resources used in this project, you can <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">delete the Google Cloud
project</a> you used for the tutorial.

Learn more about <a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">Deleting BigQuery ML models from Vertex AI Model Registry</a>

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# Delete the endpoint using the Vertex endpoint object
endpoint.undeploy_all()
endpoint.delete()

In [ ]:
# Delete BigQuery ML model

delete_query = f"""DROP MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`"""
job = bqclient.query(delete_query)
job.result()

In [ ]:
# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME